## Regret Analysis of OFL - \<Performance Experiments\>

### 1. Environment Setting

In [2]:
import  torch
import  numpy as np
import  matplotlib.pyplot as plt
from    keras import datasets
import  import_ipynb

from RegretBound import opt_param
from data import  NN5_data
from model import OFL_Model
from utils import sigma_diff

K = 1000        # Number of clients
# D = 34826       # number of parameters (MNIST)
# D = 4353        # (Air, Cond, Tom)
#D = 4548         # (Room)
P = 0.01        # Com. overhead reduction rate from FedOGD
L = 8           # Local training time (Batch Size)

# s, _, b, p = opt_param(P, D, show_result=False)
# p2 = p * L
# print(s, b, p, p2)

### 2. Experiment Setting

In [3]:
#NN5
train, test = NN5_data()
task = 'time'

print(len(train), len(test))

734 56


In [4]:
print(test[0])

[19.94  12.273 10.573 14.073 10.728 11.749 16.695  8.206  9.566 11.465
 12.727  9.028 12.684 12.854 11.069 10.573 15.249 19.118 12.613  9.864
 12.684 12.628  5.944 20.876 11.409 13.743  4.11   9.34  13.039  7.823
  9.552 14.654  8.829 24.646 11.508  5.952 14.215 21.287 12.585 11.905
 12.769 11.876 18.197 13.109  9.198  9.099 19.94  16.185  8.702 12.727
  9.442 13.204  8.39  12.23   6.931  6.944 14.821 10.429  7.299 13.138
  9.1   10.008 11.06   8.956 16.202 17.149 20.41  37.835 10.521  8.89
  9.771 10.863 14.479  9.258 13.546 12.927 13.177 13.361  9.666 12.322
  4.629 10.06  18.161 13.204  9.311  8.548 11.783  7.851 12.625 10.784
 10.258 10.31  20.213 15.742 17.254 18.359  4.945 14.085 11.573 12.296
 18.835 15.079  6.193 17.404 13.152 10.87  11.848 10.714 14.598 15.306
  9.666]


In [19]:
from statsmodels.tsa.ar_model import AutoReg

model = AutoReg(train[:,0].transpose(), 8, missing='drop')
model_fit = model.fit()

print(model_fit.y.L1)

AttributeError: 'AutoRegResults' object has no attribute 'y'

In [ ]:
Model_list = []

Model_list.append(OFL_Model('FedOGD', task, K, [False, 0, 0], 1, 1, input_size))
Model_list.append(OFL_Model('OFedAvg', task, K, [False, 0, 0], P, 1, input_size))
Model_list.append(OFL_Model('FedOMD', task, K, [False, 0, 0], 1, int(1/P), input_size))
Model_list.append(OFL_Model('OFedIQ', task, K, [True, s, b], p, 1, input_size))
#Model_list.append(OFL_Model('OFedIQ(L=8)', task, K, [True, s, b], p2, L, input_size))


2023-01-06 02:36:14.784605: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 3. Training

In [ ]:
i_max = len(y_train) // K
iter_max = 20
print(iter_max, i_max)

#iter = 15,60 / 80,4 / 40, 9

20 18


In [ ]:
for model in Model_list:
    for iter in range(iter_max):
        for i in range(i_max):
            model.train(x_train[K*i : K*(i+1)], y_train[K*i : K*(i+1)], ((i_max * iter) + (i+1)) % model.L)
        print(iter, end =',')
    print()

result_list = []
for model in Model_list:
    result_list.append(model.pull_result())

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,
0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,
0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,
0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,


### 3. Save Result

In [ ]:
import pickle

#Code: data / data_90
code = input("Code: ")

for i, model in enumerate(Model_list):
    name = model.name
    with open(f"./result/{task}_{name}_{code}.pkl","wb") as f:
        pickle.dump(result_list[i], f)

### 4. Plot Result

In [ ]:
import pickle
import matplotlib.pyplot  as plt

code = input('Code:')

with open(f"./result/{task}_FedOGD_{code}.pkl", "rb") as f:
    r1 = pickle.load(f)
with open(f"./result/{task}_OFedAvg_{code}.pkl", "rb") as f:
    r2 = pickle.load(f)
with open(f"./result/{task}_FedOMD_{code}.pkl", "rb") as f:
    r3 = pickle.load(f)
with open(f"./result/{task}_OFedIQ_{code}.pkl", "rb") as f:
    r4 = pickle.load(f)

l1 = list(range(len(r1))) 
l2 = list(range(len(r2)))
l3 = list(range(len(r3)))
l4 = list(range(len(r4)))

plt.plot(l1, r1, 'black', label=r'FedOGD')
plt.plot(l2, r2, 'red', label=r'OFedAvg ($\mathdefault{p=0.1}$)')
plt.plot(l3, r3, 'blue', label=r'FedOMD ($\mathdefault{L=10}$)')
plt.plot(l4, r4, 'green', label=r'OFedIQ ($\mathdefault{p=0.5151}$)')

plt.xlabel('time step (t)')

if task == 'clf':
    plt.ylabel('Accuracy (t)')
elif task == 'reg':
    plt.ylabel('MSE (t)')
    #plt.yscale("log")

plt.rcParams["font.family"] = "Times New Roman"
plt.grid()
plt.legend()

plt.savefig(f'./Figures/{task}_{code}.png', dpi=200, facecolor="white")
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: './result/reg_FedOGD_Energy.pkl'